CHSH(Clauser, Horne, Shimony, and Holt) inequality in action, exmaple coded in Qiskit 

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector, plot_state_qsphere
import numpy as np
from numpy.random import randint

This program implements a program for excersize mentioned in qiksit CHSH demo page at 
https://qiskit.org/textbook/ch-demos/chsh.html

It is an attemp to prove tha quantum tools enable us to go beyond the reach of classical algorithms.

IN the given Game Alice and Bob are put in separate rooms and each is given a bit 'X' and 'Y' respectively. These bits are chosen at random and independently of each other. On receiving the bit each of them replies with a bit of their own,  
'a' and 'b' Now, Winning condition Alice and Bob is - 
    'a' NOT-Equals 'b'  whenever X = Y = 1
    'a' EQUALs 'b'  if NOT(X = Y = 1). 

We will first get the best classical algorithm that given Alice and Bob a chance to win 75% of times. 


In [2]:
# first generate N pairs of random numbers X,Y   
N = 1000
bits_array = randint(0, 2, size=(N,2))

Given two random bits 'X' and 'Y' respectively to Alice and Bob, in order to maximize thier probability to win the game, Alice and Bob decides to always respond with a = b, that way they will win the game for approx. 75% of times 

In [3]:
a = 1
b = 1
result = 0
for i in range(N) :
    X = bits_array[i][0]
    Y = bits_array[i][1]
    # check for winning conditions
    if X == 1 and Y == 1 and a != b :
        result += 1    # increase the Win counter
    if not(X == 1 and Y == 1) and a == b :
        result += 1    # increase the Win counter

print(result)

766


Let's see how Quantum tools can increas chances of winning this game for Alice and Bob,  given they are allowed to share a qubit from a maximally entangled two-qubit system 1/√2(|0⟩ +|1⟩). The below program describes an strategy that Alice and Bob takes to increase their chances to win game, here they obtain the responding bits 'a' and 'b' by measuring thier entangled qubit.

They decides beforehand that Alice will only measure her qubit in eiter Z OR X basis and Bob will only measure his qubit in eiter Z+X OR Z-X basis, Also they decides that they will use bits X (Alice) and Y (Bob) to chose what basis is used to measure thier qubit.

for X=0 Alice measures her entagled qubit along the Z axis and for X=1, she measures her entagled qubit along the X axis
for Y=0 Bob measures his entagled qubit along the Z+X axis and Y=1 Bob measures his entagled qubit along the Z-X axis.


In [4]:
# Let's see how this new strategy help alice and bob win more.

a = 0
b = 0
result = 0  # Set result counter to zero
for i in range(N) :
    # Alice will be given X bit and Bob will be given Y bit
    X = bits_array[i][0]
    Y = bits_array[i][1]
    
    # build a quantumcircuit to generate maximally entangled qubit pair  
    qc = QuantumCircuit(2, 2) 
    qc.h(0)
    qc.cx(0,1)
    
    # based on a predefined strategy Alice will only measure her qubit in eiter Z OR X basis
    if X == 0 :
        qc.i(1)  # For X=0 Alice measures her entagled qubit along the Z axis 
    if X == 1 : 
        qc.h(1)  # For X=1 Alice measures her entagled qubit along the X axis 

    # based on a predefined strategy Bob will only measure his qubit in eiter Z+X OR Z-X basis
    if Y == 0 :
        qc.ry((-np.pi/4), 0)  # For Y=0 Bob measures his entagled qubit along the Z+X axis 
    if Y == 1 : 
        qc.ry((np.pi/4), 0)  # For Y=1 Bob measures his entagled qubit along the Z-X axis 

    qc.measure([0,1], [0,1])
    bkend = Aer.get_backend('qasm_simulator')
    job = execute(qc, bkend, shots=1)
    counts = job.result().get_counts(qc)
#    print(counts)
    retvalues = list(counts.keys())[0]
    a, b = retvalues[0], retvalues[1]
 
    # Check winning conditions   
    if X == 1 and Y == 1 and a != b :
        result += 1          # add 1 to win count
    if not(X == 1 and Y == 1) and a == b :
        result += 1          # add 1 to win count

print(result)   

852


Hurray!! Alice and Bob have increased their chances to WIN game by almost 10% (85%)

Below I have created four routines that may help understand usage of understand different measurement basis 

In [5]:
def Meas_for_00():
    qr = QuantumRegister(2)
    cr = ClassicalRegister(2)
    qc_00 = QuantumCircuit(qr, cr)
    qc_00.h(0)
    qc_00.cx(0,1)  # create maximally entangled state with two qubits

    # use two measurement basis apart by 45 degrees on bloch sphere give rise to obtain equal outputs 85% times 
    qc_00.ry((-np.pi/4), 0)  # For X=0 Bob measures his entagled qubit along the Z+X axis 
    #qc_00.h(1) # comment Hadamard as for Y = 0 Alice measures her entangled qubit along the Z axis
    qc_00.measure([0,1], [0,1])
    bkend = Aer.get_backend('qasm_simulator')
    job = execute(qc_00, bkend, shots=shots)
    counts = job.result().get_counts(qc_00)
    retvalues = list(counts.keys())[0]
    print(retvalues[0], retvalues[1])
    print(counts)
    return retvalues[0], retvalues[1]

In [6]:
def Meas_for_01():
    qr = QuantumRegister(2)
    cr = ClassicalRegister(2)
    qc_01 = QuantumCircuit(qr, cr)
    qc_01.h(0)
    qc_01.cx(0,1)  # create maximally entangled state with two qubits
    
    # use two measurement basis apart by 45 degrees on bloch sphere give rise to obtain equal outputs 85% times 
    qc_01.ry((-np.pi/4), 0)  # For X=1 Bob measures his entagled qubit along the Z+X axis 
    qc_01.h(1) # for Y = 0 Alice measures her entangled qubit along the X axis
    qc_01.measure([0,1], [0,1])
    bkend = Aer.get_backend('qasm_simulator')
    job = execute(qc_01, bkend, shots=shots)
    counts = job.result().get_counts(qc_01)
    retvalues = list(counts.keys())[0]
    print(retvalues[0], retvalues[1])
    print(counts)
    return retvalues[0], retvalues[1]

In [7]:
def Meas_for_10():
    qr = QuantumRegister(2)
    cr = ClassicalRegister(2)
    qc_10 = QuantumCircuit(qr, cr)
    qc_10.h(0)
    qc_10.cx(0,1)  # create maximally entangled state with two qubits

    # use two measurement basis apart by 45 degrees on bloch sphere give rise to obtain equal outputs 85% times 
    qc_10.ry((np.pi/4), 0)  # For X=0 Bob measures his entagled qubit along the Z-X axis 
    #qc_01.h(1) # comment Hadamard as for Y = 1 Alice measures her entangled qubit along the Z axis
    qc_10.measure([0,1], [0,1])
    bkend = Aer.get_backend('qasm_simulator')
    job = execute(qc_10, bkend, shots=shots)
    counts = job.result().get_counts(qc_10)
    retvalues = list(counts.keys())[0]
    print(retvalues[0], retvalues[1])
    print(counts)
    return retvalues[0], retvalues[1]

In [8]:
def Meas_for_11():
    qr = QuantumRegister(2)
    cr = ClassicalRegister(2)
    qc_11 = QuantumCircuit(qr, cr)
    qc_11.h(0)
    qc_11.cx(0,1)   # create maximally entangled state with two qubits
    
    # use two measurement basis apart by 135 degrees on bloch sphere give rise to opposite outputs 85% times
    qc_11.ry(np.pi/4, 0)  # For X=1  Bob measures entagled qubit along the Z-X axis 
    qc_11.h(1) # For Y = 1 Alice measures entagled qubit along the X axis
    qc_11.measure([0,1], [0,1])
    bkend = Aer.get_backend('qasm_simulator')
    job = execute(qc_11, bkend, shots=shots)
    counts = job.result().get_counts(qc_11)
    retvalues = list(counts.keys())[0]
    print(retvalues[0], retvalues[1])
    print(counts)
    return retvalues[0], retvalues[1]

In [9]:
# print differnt results, if you want to increase the number of shots then return the dict key with maximum count
shots = 1
print(Meas_for_00())
print(Meas_for_01())
print(Meas_for_10())
print(Meas_for_11())

0 0
{'00': 1}
('0', '0')
0 0
{'00': 1}
('0', '0')
1 1
{'11': 1}
('1', '1')
0 1
{'01': 1}
('0', '1')


In [ ]:
# end of program